In [1]:
from random import random, seed
import numpy as np
from random import randrange
from csv import reader

In [2]:
# creation of the architecture.
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [44]:
# activation = X1*W1 + X2*W2 + ... + Xn*Wn + bias.
def activate(weights, inputs):
    
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [5]:
# choose the activation function, sigmoid is the function by default.
def transfer(activation, function):
    
    if function == 'tanh':
        return (np.exp(activation) - np.exp(-activation)) / (np.exp(activation) + np.exp(-activation))
    elif function == 'relu':
        if activation > 0:
            return activation
        else:
            return 0
    elif function == 'arctan':
        return np.arctan(activation)
    elif function == 'softplus':
        return np.log(1 + np.exp(activation))
    else:
        return 1 / (1 + np.exp(-activation))

In [6]:
# compute of the outputs.
def forward_propagate(network, row, function):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['activation'] = activation
            neuron['output'] = transfer(activation, function)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [9]:
# there is the derivative of the functions use for the activation. See: https://en.wikipedia.org/wiki/Activation_function
def transfer_derivative(output, function, activation):
    
    if function == 'tanh':
        return 1 - output**2
    elif function == 'relu':
        if activation > 0:
            return 1
        else:
            return 0
    elif function == 'arctan':
        return 1/(activation**2 + 1)
    elif function == 'softplus':
        return 1/(1 + np.exp(-activation))
    else:
        return output * (1 - output)

In [45]:
# compute the errors in function of the expected outputs.
def backward_propagate_error(network, expected, function):
    
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)): 
                error = 0
                for neuron in network[i + 1]: 
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], function,neuron['activation'])

In [14]:
# change the value of the weights after the back-propagation.
def update_weights(network, row, l_rate):
    
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [46]:
# application of the algorithm for each value of a dataset. It's repeat several time.
def train_network(network, train, l_rate, n_epoch, n_outputs,function):
    
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row, function)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected, function)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        

In [18]:
# it permits us to compute some data to get a result in function of a network.
def predict(network, row, function):
    
    outputs = forward_propagate(network, row, function)
    return outputs.index(max(outputs))

In [23]:
# loading a csv file.
def load_csv(filename):
    
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [24]:
# convert string column to float.
def str_column_to_float(dataset, column):
    
    for row in dataset:
        row[column] = float(row[column].strip())

In [25]:
# convert string column to integer.
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [26]:
# find the min and max values for each column.
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

In [27]:
# rescale dataset columns to the range 0-1.
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [28]:
# split a dataset into k folds.
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [29]:
# calculate accuracy percentage.
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [30]:
# evaluate an algorithm using a cross validation split.
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [31]:
# backpropagation Algorithm With Stochastic Gradient Descent.
def back_propagation(train, test, l_rate, n_epoch, n_hidden, function):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs, function)
    predictions = list()
    for row in test:
        prediction = predict(network, row, function)
        predictions.append(prediction)
    return(predictions)

In [32]:
# set a seed for the random.
seed(60198)

In [33]:
filename = 'webData.csv'
dataset = load_csv(filename)

In [34]:
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)

In [36]:
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

In [40]:
# allows us to see 
def trainings(n_folds, l_rate, n_epoch, n_hidden, function):
    
    scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden, function)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    print('Activation function: %s' % function)

In [41]:
trainings(5, 0.3, 500, 3, 'solftplus')

>epoch=0, lrate=0.300, error=122.262
>epoch=1, lrate=0.300, error=107.196
>epoch=2, lrate=0.300, error=98.311
>epoch=3, lrate=0.300, error=86.978
>epoch=4, lrate=0.300, error=77.195
>epoch=5, lrate=0.300, error=70.521
>epoch=6, lrate=0.300, error=66.342
>epoch=7, lrate=0.300, error=63.638
>epoch=8, lrate=0.300, error=61.706
>epoch=9, lrate=0.300, error=60.149
>epoch=10, lrate=0.300, error=58.707
>epoch=11, lrate=0.300, error=57.179
>epoch=12, lrate=0.300, error=55.425
>epoch=13, lrate=0.300, error=53.379
>epoch=14, lrate=0.300, error=50.986
>epoch=15, lrate=0.300, error=48.209
>epoch=16, lrate=0.300, error=45.132
>epoch=17, lrate=0.300, error=41.971
>epoch=18, lrate=0.300, error=38.977
>epoch=19, lrate=0.300, error=36.313
>epoch=20, lrate=0.300, error=34.033
>epoch=21, lrate=0.300, error=32.119
>epoch=22, lrate=0.300, error=30.517
>epoch=23, lrate=0.300, error=29.172
>epoch=24, lrate=0.300, error=28.031
>epoch=25, lrate=0.300, error=27.053
>epoch=26, lrate=0.300, error=26.206
>epoch=27

>epoch=224, lrate=0.300, error=10.869
>epoch=225, lrate=0.300, error=10.855
>epoch=226, lrate=0.300, error=10.841
>epoch=227, lrate=0.300, error=10.826
>epoch=228, lrate=0.300, error=10.812
>epoch=229, lrate=0.300, error=10.798
>epoch=230, lrate=0.300, error=10.785
>epoch=231, lrate=0.300, error=10.771
>epoch=232, lrate=0.300, error=10.757
>epoch=233, lrate=0.300, error=10.743
>epoch=234, lrate=0.300, error=10.730
>epoch=235, lrate=0.300, error=10.716
>epoch=236, lrate=0.300, error=10.703
>epoch=237, lrate=0.300, error=10.689
>epoch=238, lrate=0.300, error=10.676
>epoch=239, lrate=0.300, error=10.662
>epoch=240, lrate=0.300, error=10.649
>epoch=241, lrate=0.300, error=10.635
>epoch=242, lrate=0.300, error=10.622
>epoch=243, lrate=0.300, error=10.609
>epoch=244, lrate=0.300, error=10.595
>epoch=245, lrate=0.300, error=10.582
>epoch=246, lrate=0.300, error=10.568
>epoch=247, lrate=0.300, error=10.555
>epoch=248, lrate=0.300, error=10.541
>epoch=249, lrate=0.300, error=10.528
>epoch=250, 

>epoch=449, lrate=0.300, error=7.686
>epoch=450, lrate=0.300, error=7.677
>epoch=451, lrate=0.300, error=7.668
>epoch=452, lrate=0.300, error=7.659
>epoch=453, lrate=0.300, error=7.650
>epoch=454, lrate=0.300, error=7.641
>epoch=455, lrate=0.300, error=7.632
>epoch=456, lrate=0.300, error=7.624
>epoch=457, lrate=0.300, error=7.615
>epoch=458, lrate=0.300, error=7.606
>epoch=459, lrate=0.300, error=7.597
>epoch=460, lrate=0.300, error=7.589
>epoch=461, lrate=0.300, error=7.580
>epoch=462, lrate=0.300, error=7.571
>epoch=463, lrate=0.300, error=7.563
>epoch=464, lrate=0.300, error=7.554
>epoch=465, lrate=0.300, error=7.546
>epoch=466, lrate=0.300, error=7.538
>epoch=467, lrate=0.300, error=7.529
>epoch=468, lrate=0.300, error=7.521
>epoch=469, lrate=0.300, error=7.512
>epoch=470, lrate=0.300, error=7.504
>epoch=471, lrate=0.300, error=7.496
>epoch=472, lrate=0.300, error=7.488
>epoch=473, lrate=0.300, error=7.480
>epoch=474, lrate=0.300, error=7.471
>epoch=475, lrate=0.300, error=7.463
>

>epoch=175, lrate=0.300, error=9.427
>epoch=176, lrate=0.300, error=9.404
>epoch=177, lrate=0.300, error=9.382
>epoch=178, lrate=0.300, error=9.360
>epoch=179, lrate=0.300, error=9.338
>epoch=180, lrate=0.300, error=9.316
>epoch=181, lrate=0.300, error=9.294
>epoch=182, lrate=0.300, error=9.273
>epoch=183, lrate=0.300, error=9.252
>epoch=184, lrate=0.300, error=9.231
>epoch=185, lrate=0.300, error=9.210
>epoch=186, lrate=0.300, error=9.190
>epoch=187, lrate=0.300, error=9.169
>epoch=188, lrate=0.300, error=9.149
>epoch=189, lrate=0.300, error=9.129
>epoch=190, lrate=0.300, error=9.110
>epoch=191, lrate=0.300, error=9.090
>epoch=192, lrate=0.300, error=9.071
>epoch=193, lrate=0.300, error=9.052
>epoch=194, lrate=0.300, error=9.033
>epoch=195, lrate=0.300, error=9.014
>epoch=196, lrate=0.300, error=8.996
>epoch=197, lrate=0.300, error=8.977
>epoch=198, lrate=0.300, error=8.959
>epoch=199, lrate=0.300, error=8.941
>epoch=200, lrate=0.300, error=8.924
>epoch=201, lrate=0.300, error=8.906
>

>epoch=404, lrate=0.300, error=6.450
>epoch=405, lrate=0.300, error=6.439
>epoch=406, lrate=0.300, error=6.428
>epoch=407, lrate=0.300, error=6.417
>epoch=408, lrate=0.300, error=6.407
>epoch=409, lrate=0.300, error=6.396
>epoch=410, lrate=0.300, error=6.385
>epoch=411, lrate=0.300, error=6.375
>epoch=412, lrate=0.300, error=6.364
>epoch=413, lrate=0.300, error=6.354
>epoch=414, lrate=0.300, error=6.343
>epoch=415, lrate=0.300, error=6.333
>epoch=416, lrate=0.300, error=6.323
>epoch=417, lrate=0.300, error=6.312
>epoch=418, lrate=0.300, error=6.302
>epoch=419, lrate=0.300, error=6.292
>epoch=420, lrate=0.300, error=6.282
>epoch=421, lrate=0.300, error=6.272
>epoch=422, lrate=0.300, error=6.262
>epoch=423, lrate=0.300, error=6.252
>epoch=424, lrate=0.300, error=6.242
>epoch=425, lrate=0.300, error=6.232
>epoch=426, lrate=0.300, error=6.223
>epoch=427, lrate=0.300, error=6.213
>epoch=428, lrate=0.300, error=6.203
>epoch=429, lrate=0.300, error=6.194
>epoch=430, lrate=0.300, error=6.184
>

...

>epoch=368, lrate=0.300, error=7.597
>epoch=369, lrate=0.300, error=7.591
>epoch=370, lrate=0.300, error=7.585
>epoch=371, lrate=0.300, error=7.579
>epoch=372, lrate=0.300, error=7.573
>epoch=373, lrate=0.300, error=7.567
>epoch=374, lrate=0.300, error=7.561
>epoch=375, lrate=0.300, error=7.555
>epoch=376, lrate=0.300, error=7.549
>epoch=377, lrate=0.300, error=7.543
>epoch=378, lrate=0.300, error=7.538
>epoch=379, lrate=0.300, error=7.532
>epoch=380, lrate=0.300, error=7.526
>epoch=381, lrate=0.300, error=7.520
>epoch=382, lrate=0.300, error=7.514
>epoch=383, lrate=0.300, error=7.509
>epoch=384, lrate=0.300, error=7.503
>epoch=385, lrate=0.300, error=7.497
>epoch=386, lrate=0.300, error=7.492
>epoch=387, lrate=0.300, error=7.486
>epoch=388, lrate=0.300, error=7.480
>epoch=389, lrate=0.300, error=7.475
>epoch=390, lrate=0.300, error=7.469
>epoch=391, lrate=0.300, error=7.463
>epoch=392, lrate=0.300, error=7.458
>epoch=393, lrate=0.300, error=7.452
>epoch=394, lrate=0.300, error=7.447
>

>epoch=99, lrate=0.300, error=12.507
>epoch=100, lrate=0.300, error=12.455
>epoch=101, lrate=0.300, error=12.403
>epoch=102, lrate=0.300, error=12.353
>epoch=103, lrate=0.300, error=12.303
>epoch=104, lrate=0.300, error=12.254
>epoch=105, lrate=0.300, error=12.206
>epoch=106, lrate=0.300, error=12.159
>epoch=107, lrate=0.300, error=12.112
>epoch=108, lrate=0.300, error=12.066
>epoch=109, lrate=0.300, error=12.021
>epoch=110, lrate=0.300, error=11.976
>epoch=111, lrate=0.300, error=11.932
>epoch=112, lrate=0.300, error=11.888
>epoch=113, lrate=0.300, error=11.845
>epoch=114, lrate=0.300, error=11.803
>epoch=115, lrate=0.300, error=11.761
>epoch=116, lrate=0.300, error=11.720
>epoch=117, lrate=0.300, error=11.679
>epoch=118, lrate=0.300, error=11.639
>epoch=119, lrate=0.300, error=11.599
>epoch=120, lrate=0.300, error=11.560
>epoch=121, lrate=0.300, error=11.521
>epoch=122, lrate=0.300, error=11.483
>epoch=123, lrate=0.300, error=11.445
>epoch=124, lrate=0.300, error=11.408
>epoch=125, l

>epoch=327, lrate=0.300, error=7.166
>epoch=328, lrate=0.300, error=7.154
>epoch=329, lrate=0.300, error=7.142
>epoch=330, lrate=0.300, error=7.130
>epoch=331, lrate=0.300, error=7.119
>epoch=332, lrate=0.300, error=7.107
>epoch=333, lrate=0.300, error=7.096
>epoch=334, lrate=0.300, error=7.084
>epoch=335, lrate=0.300, error=7.073
>epoch=336, lrate=0.300, error=7.061
>epoch=337, lrate=0.300, error=7.050
>epoch=338, lrate=0.300, error=7.039
>epoch=339, lrate=0.300, error=7.028
>epoch=340, lrate=0.300, error=7.016
>epoch=341, lrate=0.300, error=7.005
>epoch=342, lrate=0.300, error=6.994
>epoch=343, lrate=0.300, error=6.983
>epoch=344, lrate=0.300, error=6.973
>epoch=345, lrate=0.300, error=6.962
>epoch=346, lrate=0.300, error=6.951
>epoch=347, lrate=0.300, error=6.940
>epoch=348, lrate=0.300, error=6.930
>epoch=349, lrate=0.300, error=6.919
>epoch=350, lrate=0.300, error=6.908
>epoch=351, lrate=0.300, error=6.898
>epoch=352, lrate=0.300, error=6.887
>epoch=353, lrate=0.300, error=6.877
>

>epoch=57, lrate=0.300, error=17.257
>epoch=58, lrate=0.300, error=17.153
>epoch=59, lrate=0.300, error=17.052
>epoch=60, lrate=0.300, error=16.951
>epoch=61, lrate=0.300, error=16.851
>epoch=62, lrate=0.300, error=16.752
>epoch=63, lrate=0.300, error=16.653
>epoch=64, lrate=0.300, error=16.555
>epoch=65, lrate=0.300, error=16.458
>epoch=66, lrate=0.300, error=16.361
>epoch=67, lrate=0.300, error=16.265
>epoch=68, lrate=0.300, error=16.169
>epoch=69, lrate=0.300, error=16.074
>epoch=70, lrate=0.300, error=15.979
>epoch=71, lrate=0.300, error=15.884
>epoch=72, lrate=0.300, error=15.791
>epoch=73, lrate=0.300, error=15.697
>epoch=74, lrate=0.300, error=15.605
>epoch=75, lrate=0.300, error=15.513
>epoch=76, lrate=0.300, error=15.422
>epoch=77, lrate=0.300, error=15.331
>epoch=78, lrate=0.300, error=15.241
>epoch=79, lrate=0.300, error=15.152
>epoch=80, lrate=0.300, error=15.064
>epoch=81, lrate=0.300, error=14.976
>epoch=82, lrate=0.300, error=14.890
>epoch=83, lrate=0.300, error=14.804
>

>epoch=285, lrate=0.300, error=7.877
>epoch=286, lrate=0.300, error=7.866
>epoch=287, lrate=0.300, error=7.855
>epoch=288, lrate=0.300, error=7.844
>epoch=289, lrate=0.300, error=7.833
>epoch=290, lrate=0.300, error=7.822
>epoch=291, lrate=0.300, error=7.811
>epoch=292, lrate=0.300, error=7.800
>epoch=293, lrate=0.300, error=7.789
>epoch=294, lrate=0.300, error=7.778
>epoch=295, lrate=0.300, error=7.768
>epoch=296, lrate=0.300, error=7.757
>epoch=297, lrate=0.300, error=7.746
>epoch=298, lrate=0.300, error=7.736
>epoch=299, lrate=0.300, error=7.726
>epoch=300, lrate=0.300, error=7.715
>epoch=301, lrate=0.300, error=7.705
>epoch=302, lrate=0.300, error=7.695
>epoch=303, lrate=0.300, error=7.685
>epoch=304, lrate=0.300, error=7.675
>epoch=305, lrate=0.300, error=7.665
>epoch=306, lrate=0.300, error=7.655
>epoch=307, lrate=0.300, error=7.645
>epoch=308, lrate=0.300, error=7.635
>epoch=309, lrate=0.300, error=7.625
>epoch=310, lrate=0.300, error=7.616
>epoch=311, lrate=0.300, error=7.606
>